In [45]:
import argparse
import numpy as np
import os
import os.path as osp 
import json
import shutil

import lib 
from lib.taxonomy import loading

In [20]:
# parser = argparse.ArgumentParser(description='Arguments for production partition')
# parser.add_argument('--dataset_directory', type=str)
# parser.add_argument('--new_dir', type=str)
# parser.add_argument('--meta_file', type=str, help='The meta.json to load from.')
# args = parser.parse_args()

dataset_directory = '/ssd/esteva/skindata4/splits/nine-way/val/inflammatory'
new_dir = dataset_directory + '-split'
meta_file = '/ssd/esteva/skindata4/meta.json'

image_names = os.listdir(dataset_directory)
print len(image_names)

meta = json.load(open(meta_file))
meta = [m for m in meta if 'tax_path' in m]
filename2meta = loading.Field2meta(meta, field='filename')
print 'meta created'

10666
meta created


In [48]:
def full_tax_path(entry):
    """Returns i.e. acne_of-the-leg_but-not-the-ass"""
    tax_path = entry['tax_path'][1:]
    tax_path = '_'.join(['-'.join(l) for l in tax_path])
    return tax_path

def copy(src, dst):
    """Copy src to dst, making a new symlink if src is a symlink."""
    if os.path.islink(src):
        linkto = os.readlink(src)
        os.symlink(linkto, dst)
    else:
        shutil.copy(src,dst)

In [40]:
new_dir_structure = {}
bad = 0
for filename in image_names:
    meta_entries = filename2meta(filename)
    tax_paths = [full_tax_path(m) for m in meta_entries]
    if len(tax_paths) > 1:
        bad += 1
    tax_path = tax_paths[0]
    if tax_path in new_dir_structure:
        new_dir_structure[tax_path].append(filename)
    else:
        new_dir_structure[tax_path] = [filename]

In [42]:
dataset_directory

'/ssd/esteva/skindata4/splits/nine-way/val/inflammatory'

In [50]:

if osp.exists(new_dir):
    print 'Deleting previous'
    shutil.rmtree(new_dir)
os.makedirs(new_dir)

for new_class in new_dir_structure:
    print '\rMaking: %s' % new_class,
    subdir = osp.join(new_dir, new_class)
    os.makedirs(subdir)
    for filename in new_dir_structure[new_class]:
        src = osp.join(dataset_directory, filename)
        dst = osp.join(subdir, filename)
        copy(src, dst)        

Deleting previous
Making: skin-diseases-of-pregnancy_pruritic-urticarial-papules-and-plaques-of-pregnancy
